In [1]:
from tsfresh.examples import load_robot_execution_failures
import pandas as pd
import tsfresh
import matplotlib.pyplot as plt

from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from ML.feature_selection import FeatureSelection
from ML.model import Model, min_acc
from ML.process_data import Data
from utils.help_classes import ScalerEnum, ModelsEnum
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler

from utils.help_classes import ModelsEnum
from sklearn.model_selection import train_test_split
import os
import datetime
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

import pickle
import numpy as np
import category_encoders as ce
import plotly_express as px
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from numpy.linalg import LinAlgError

## Split data into 3 datasets

In [50]:
X = pd.read_csv(os.path.join('data_csv', 'features_train.csv'))

X['time_stamp'] = pd.to_datetime(X['time_stamp'])

In [51]:
X_train_1 = X.loc[X['time_stamp'] < X['time_stamp'].quantile(q=0.9)]

X_others = X.loc[X['time_stamp'] >= X['time_stamp'].quantile(q=0.9)]
X_train_2 = X_others.loc[X_others['time_stamp'] < X_others['time_stamp'].quantile(q=0.5)]
X_val = X_others.loc[X_others['time_stamp'] >= X_others['time_stamp'].quantile(q=0.5)]

In [4]:
print('X_train_1', X_train_1.shape)
print('X_train_2', X_train_2.shape)
print('X_val', X_val.shape)

X_train_1 (29466, 56)
X_train_2 (1638, 56)
X_val (1637, 56)


In [5]:
assert X_train_1.shape[0] + X_train_2.shape[0] + X_val.shape[0] == X.shape[0]

## Train first layer

In [6]:
y_train_1 = X_train_1['label']
X_train_1 = X_train_1.drop(columns=['label', 'time_stamp', 'traj'])

y_val = X_val['label']
X_val = X_val.drop(columns=['label', 'time_stamp', 'traj'])

In [7]:
print('before sampling', X_train_1.shape)
X_train_1 = X_train_1.fillna(X_train_1.mean())

sampling_strategy = "not minority"
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
X_train_1, y_train_1 = rus.fit_resample(X_train_1, y_train_1)
print('after sampling', X_train_1.shape)

before sampling (29466, 53)
after sampling (6381, 53)


In [8]:
X_train_1 = X_train_1.fillna(X_train_1.median())
X_val = X_val.fillna(X_val.median())

In [9]:
scaler = MinMaxScaler()
scaler.fit(X_train_1)
X_train_1 = scaler.transform(X_train_1)
X_val = scaler.transform(X_val)

In [10]:
all_classifiers = [ModelsEnum.Xgboost, ModelsEnum.RF, ModelsEnum.AB]
its = [10, 100, 10]
trained_classifier = []

In [11]:
class_weight = {0: 0.3, 1: 3, 2: 2}

## Ensemble learning from 3 different models 

In [12]:
for it, cl in zip(its, all_classifiers):
    ml_model = Model(model_name="time_series_3_" + str(cl), classifier_method=cl,  verbose=True, binary_task=False, 
                     write_csv=True, class_weight=class_weight)
    trained_model, results_train = ml_model.cross_fold_validation(X_train_1, y_train_1, grid_search=False, cv=3, 
                                                                  n_iter=it, scoring=make_scorer(min_acc), n_jobs=2)
    trained_classifier.append(trained_model)
    results_dict = ml_model.test_set(X_val, y_val)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


{'accuracy': [0.9651802076970067], 'f1_': [0.9599761905386699], 'jaccard': [0.9327036599763873], 'recall': [0.9651802076970067], 'min_acc': [0.9336870026525199], 'acc_0': [0.9727195225916454], 'acc_1': [0.9336870026525199], 'acc_2': [1.0]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'accuracy': [0.938912645082468], 'f1_': [0.8773154928075423], 'jaccard': [0.8848589522164652], 'recall': [0.938912645082468], 'min_acc': [0.8673740053050398], 'acc_0': [0.9616368286445013], 'acc_1': [0.8673740053050398], 'acc_2': [0.9425287356321839]}
Fitting 3 folds for each of 10 candidates, totalling 30 fits


One or more of the test scores are non-finite: [0.8232252  0.04795487 0.82933709        nan        nan 0.84955336
 0.88293371 0.83591913 0.         0.89938881]
One or more of the train scores are non-finite: [0.97696286 0.05030559 0.8370945         nan        nan 0.85848613
 0.98330983 0.98142924 0.         1.        ]


{'accuracy': [0.9712889431887599], 'f1_': [0.952411908880618], 'jaccard': [0.9441805225653207], 'recall': [0.9712889431887599], 'min_acc': [0.9336870026525199], 'acc_0': [0.9820971867007673], 'acc_1': [0.9336870026525199], 'acc_2': [0.9885057471264368]}


## Ensemble learning from trees of random forest

In [32]:
ml_model = Model(model_name="time_series_3_" + str(ModelsEnum.RF), classifier_method=ModelsEnum.RF,  verbose=True, 
                 binary_task=False,  write_csv=True, class_weight=class_weight)

trained_model, results_train = ml_model.cross_fold_validation(X_train_1, y_train_1, grid_search=False, cv=3, 
                                                              n_iter=100, scoring=make_scorer(min_acc), n_jobs=2)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [33]:
results_dict = ml_model.test_set(X_val, y_val)

{'accuracy': [0.938912645082468], 'f1_': [0.8773154928075423], 'jaccard': [0.8848589522164652], 'recall': [0.938912645082468], 'min_acc': [0.8673740053050398], 'acc_0': [0.9616368286445013], 'acc_1': [0.8673740053050398], 'acc_2': [0.9425287356321839]}


In [38]:
predictions_train_2 = np.concatenate(list(map(lambda x: x.predict_proba(X_train_2), 
                                              trained_model.best_estimator_.estimators_)), axis=-1)
predictions_val = np.concatenate(list(map(lambda x: x.predict_proba(X_val), trained_model.best_estimator_.estimators_)), axis=-1)

print("predictions_train_2", predictions_train_2.shape)
print("predictions_val", predictions_val.shape)

predictions_train_2 (375, 120)
predictions_val (1637, 120)


## 2nd layer

In [13]:
y_train_2 = X_train_2['label']
X_train_2 = X_train_2.drop(columns=['label', 'time_stamp', 'traj'])
X_train_2 = X_train_2.fillna(X_train_2.median())
X_train_2 = scaler.transform(X_train_2)

In [14]:
print('before sampling', X_train_2.shape)

sampling_strategy = "not minority"
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)
X_train_2, y_train_2 = rus.fit_resample(X_train_2, y_train_2)
print('after sampling', X_train_2.shape)

before sampling (1638, 53)
after sampling (375, 53)


In [40]:
predictions_train_2 = np.empty(shape=(X_train_2.shape[0],0))
predictions_val = np.empty(shape=(X_val.shape[0],0))

for i, model in enumerate(trained_classifier):
    if i == 1:
        continue
    model_prediction_train_2 = model.predict_proba(X_train_2)
    predictions_train_2 = np.concatenate((predictions_train_2, model_prediction_train_2), axis=1)
    
    model_prediction_test = model.predict_proba(X_val)
    predictions_val = np.concatenate((predictions_val, model_prediction_test), axis=1)
    
print("predictions_train_2", predictions_train_2.shape)
print("predictions_val", predictions_val.shape)

predictions_train_2 (375, 6)
predictions_val (1637, 6)


Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


In [43]:
ml_model = Model(model_name="time_series_3_" + str(ModelsEnum.LR), classifier_method=ModelsEnum.LR, verbose=True, 
                 binary_task=False, write_csv=True)

trained_model, results_train = ml_model.cross_fold_validation(predictions_train_2, y_train_2, grid_search=False, cv=3,
                                                              n_iter=300, scoring=make_scorer(min_acc), n_jobs=1)
results_dict = ml_model.test_set(predictions_val, y_val)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
[CV 1/3] END C=2.782559402207126e-07, penalty=elasticnet, solver=newton-cg, tol=0.0012915496650148853; total time=   0.0s
[CV 2/3] END C=2.782559402207126e-07, penalty=elasticnet, solver=newton-cg, tol=0.0012915496650148853; total time=   0.0s
[CV 3/3] END C=2.782559402207126e-07, penalty=elasticnet, solver=newton-cg, tol=0.0012915496650148853; total time=   0.0s
[CV 1/3] END C=5.994842503189421, penalty=l1, solver=lbfgs, tol=0.35938136638046403; total time=   0.0s
[CV 2/3] END C=5.994842503189421, penalty=l1, solver=lbfgs, tol=0.35938136638046403; total time=   0.0s
[CV 3/3] END C=5.994842503189421, penalty=l1, solver=lbfgs, tol=0.35938136638046403; total time=   0.0s
[CV 1/3] END C=35938136638.0464, penalty=elasticnet, solver=newton-cg, tol=2.782559402207126e-07; total time=   0.0s
[CV 2/3] END C=35938136638.0464, penalty=elasticnet, solver=newton-cg, tol=2.782559402207126e-07; total time=   0.0s
[CV 3/3] END C=3593813663

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\jeremyl\documents\timeseriesanalysis\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\jeremyl\documents\timeseriesanalysis\venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\users\jeremyl\documents\timeseriesanalysis\venv\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got elasticnet penalty.

Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File

The max_iter was reached which means the coef_ did not converge


[CV 1/3] END C=10000000000000.0, penalty=l2, solver=sag, tol=4.641588833612782e-06; total time=   0.2s


The max_iter was reached which means the coef_ did not converge


...

The max_iter was reached which means the coef_ did not converge


[CV 2/3] END C=464158.88336127915, penalty=l1, solver=saga, tol=4.641588833612782e-06; total time=   0.3s


The max_iter was reached which means the coef_ did not converge
The line search algorithm did not converge
Line Search failed
The line search algorithm did not converge
Line Search failed
One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
 0.93573364 0.91173055        nan 0.91153697        nan        nan
 0.                nan        nan        nan 0.91947348        nan
 0.90360046        nan        nan 0.         0.90360046 0.91947348
        nan        nan 0.         0.93573364        nan 0.91966705
        nan 0.90360046 0.         0.91173055        nan 0.91947348
        nan        nan        nan        nan 0.91947348        nan
        nan        nan 0.         0.93573364 0.91173055 0.
        nan        nan 0.91947348 0.93573364        nan        nan
 0.         0.93573364        nan 0.93573364        nan 0.91947348
 0.                nan 0.91947348        nan        nan        nan
        nan        nan        nan   

[CV 3/3] END C=464158.88336127915, penalty=l1, solver=saga, tol=4.641588833612782e-06; total time=   0.3s
[CV 1/3] END C=464158.88336127915, penalty=l2, solver=newton-cg, tol=0.021544346900318867; total time=   0.0s
[CV 2/3] END C=464158.88336127915, penalty=l2, solver=newton-cg, tol=0.021544346900318867; total time=   0.0s
[CV 3/3] END C=464158.88336127915, penalty=l2, solver=newton-cg, tol=0.021544346900318867; total time=   0.0s
[CV 1/3] END C=2.782559402207126e-07, penalty=l2, solver=newton-cg, tol=2.782559402207126e-07; total time=   0.0s
[CV 2/3] END C=2.782559402207126e-07, penalty=l2, solver=newton-cg, tol=2.782559402207126e-07; total time=   0.0s
[CV 3/3] END C=2.782559402207126e-07, penalty=l2, solver=newton-cg, tol=2.782559402207126e-07; total time=   0.0s
[CV 1/3] END C=129154966.50148933, penalty=l1, solver=saga, tol=0.021544346900318867; total time=   0.0s
[CV 2/3] END C=129154966.50148933, penalty=l1, solver=saga, tol=0.021544346900318867; total time=   0.0s
[CV 3/3] END

The max_iter was reached which means the coef_ did not converge


## Test set

In [45]:
X_test = pd.read_csv(os.path.join('data_csv', 'features_test.csv'))

In [46]:
traj = X_test['traj'].values

In [19]:
X_test = X_test.drop(columns=['label', 'time_stamp', 'traj'])
X_test = X_test.fillna(X_test.median())
X_test = scaler.transform(X_test)

In [22]:
predictions_test = np.empty(shape=(X_test.shape[0],0))

for i, model in enumerate(trained_classifier):
    if i == 1:
        continue
    model_prediction_test = model.predict_proba(X_test)
    predictions_test = np.concatenate((predictions_test, model_prediction_test), axis=1)

Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption


In [23]:
final_prediction = trained_model.predict(predictions_test)

In [24]:
test_predictions = pd.DataFrame({
        'trajectory_ind': traj,
        'label': final_prediction
    })
test_predictions.to_csv('submission_3_EnsembleLearningTrees.csv', index=False)

In [44]:
final_prediction_proba = trained_model.predict_proba(predictions_test)

print(final_prediction_proba.shape)

(8185, 3)


In [31]:
np.argmax(final_prediction_proba, axis=1)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [47]:
test_predictions = pd.DataFrame({
        'trajectory_ind': traj,
        'final_prediction_proba_0': final_prediction_proba[:, 0],
        'final_prediction_proba_1': final_prediction_proba[:, 1],
        'final_prediction_proba_2': final_prediction_proba[:, 2],
    })
test_predictions.to_csv('probas_PseudoLabelling.csv', index=False)

In [48]:
val_prediction = trained_model.predict_proba(predictions_val)
print(val_prediction.shape)

(1637, 3)


In [52]:
val_csv = pd.DataFrame({
        'trajectory_ind': X_val.traj,
        'final_prediction_proba_0': val_prediction[:, 0],
        'final_prediction_proba_1': val_prediction[:, 1],
        'final_prediction_proba_2': val_prediction[:, 2],
    })
val_csv.to_csv('probas_PseudoLabelling_val.csv', index=False)